## This is the notebook for python

We are using SciKit Learn to provide us the machine learning models for earthquake prediction given a location and timestamp and in the future also predict an upcoming earthquake along with when it may occur.

We are (and have) experimented with multiple different models and datasets. We used Kaggle to provide us the datasets required. The models that we've tested are Linear Regression, Multiple Regression, Decision Tree, Random Forest, and Grid Search Via Random Forest. We are also planning on testing whether neural networks would provide us a better optimization. Currently, we are testing grid search via random forest and our highest scoring models based on the ones tested already is Random Forest with an accuracy of 86%.

In [1]:
# install dependencies: numpy, matplotlib, pandas
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install basemap
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: scikit-learn in c:\users\adity\appdata\local\programs\python\python311\lib\site-packages (1.4.0)




[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import required dependancies
#-----------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\adity\AppData\Local\Temp\ipykernel_18152\4084351598.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# get the data
data = pd.read_csv("silver.csv")
data = data[['date', 'hour', 'latitude', 'longitude', 'mag', 'depth']]
# convert date and hour to timestamp
import datetime
import time

timestamp = []
for d, t in zip(data['date'], data['hour']):
    try:
        ts = datetime.datetime.strptime(d+' '+str(t) + ':0:0', '%Y-%m-%d %H:%M:%S')
        timestamp.append(time.mktime(ts.timetuple()))
    except ValueError:
        # print('ValueError')  
        timestamp.append('ValueError')

data['timestamp'] = pd.Series(timestamp).values
data = data.drop(['date', 'hour'], axis=1)  
data = data[data.timestamp != 'ValueError']  
data.describe()

,latitude,longitude,mag,depth,timestamp
count,3.855719e+06,3.855719e+06,3.855719e+06,3.855719e+06,3.855719e+06
mean,3.619690e+01,-1.000200e+02,1.839301e+00,2.228747e+01,1.118936e+09
std,1.993314e+01,7.744373e+01,1.308862e+00,5.490393e+01,3.869948e+08
min,-8.442200e+01,-1.799990e+02,-9.990000e+00,0.000000e+00,9.471240e+07
25%,3.377566e+01,-1.410697e+02,9.600000e-01,3.317000e+00,8.341326e+08
50%,3.757433e+01,-1.189197e+02,1.500000e+00,7.500000e+00,1.190048e+09
75%,4.460733e+01,-1.159835e+02,2.380000e+00,1.520000e+01,1.454422e+09
max,8.738600e+01,1.800000e+02,9.100000e+00,7.358000e+02,1.641010e+09


In [10]:
# trunticate the data and refactor to timestamp
#-------------------------------------------------
final_data = data[data['mag'] > 0]
final_data.describe()

,latitude,longitude,mag,depth,timestamp
count,3.749279e+06,3.749279e+06,3.749279e+06,3.749279e+06,3.749279e+06
mean,3.610546e+01,-9.983861e+01,1.900921e+00,2.256044e+01,1.114155e+09
std,2.004812e+01,7.768673e+01,1.266708e+00,5.530050e+01,3.890529e+08
min,-8.442200e+01,-1.799990e+02,1.000000e-02,0.000000e+00,9.471240e+07
25%,3.368367e+01,-1.416976e+02,1.000000e+00,3.340000e+00,8.231220e+08
50%,3.756700e+01,-1.189253e+02,1.550000e+00,7.547000e+00,1.184695e+09
75%,4.451600e+01,-1.159252e+02,2.400000e+00,1.560000e+01,1.452330e+09
max,8.738600e+01,1.800000e+02,9.100000e+00,7.358000e+02,1.641010e+09


In [ ]:
# plot the values for data visualization
#----------------------------------------
from mpl_toolkits.basemap import Basemap
m = Basemap(projection='mill',llcrnrlat=-80,urcrnrlat=80, llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')

longitudes = final_data["longitude"].tolist()
latitudes = final_data["latitude"].tolist()
#m = Basemap(width=12000000,height=9000000,projection='lcc', resolution=None,lat_1=80.,lat_2=55,lat_0=80,lon_0=-107.)
x,y = m(longitudes,latitudes)
fig = plt.figure(figsize=(12,10))
plt.title("All affected areas")
m.plot(x, y, "o", markersize = 2, color = 'blue')
m.drawcoastlines()
m.fillcontinents(color='coral',lake_color='aqua')
m.drawmapboundary()
m.drawcountries()
plt.show()

In [8]:
final_data.to_csv("final_data.csv", encoding='utf-8', index=False)

In [ ]:
#from sklearn.model_selection import GridSearchCV

#parameters = {'n_estimators':[10, 20, 50, 100, 200, 500]}  
  
#grid_obj = GridSearchCV(randomforest_regressor, parameters)
#grid_fit = grid_obj.fit(X_train, y_train)
#best_fit = grid_fit.best_estimator_
#best_fit.score(X_test, y_test)